# Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras

import sys
sys.path.insert(0, '../../')
from plot_graphs import plot_loss, plot_acc

# Set image size, batch size, and import dataset

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

train = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-03 15:04:04.756843: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 15:04:04.756955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Train all except the first two conv2d layers

In [3]:
vtid = keras.models.load_model('../../models/vtid2/m2_medium.h5')
vtid.trainable = False

for layer in vtid.layers[6:]:
  layer.trainable = True

x = vtid.layers[-2].output
outputs = layers.Dense(6, activation='softmax', name='classifier')(x)
model = keras.Model(vtid.inputs, outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                         

In [4]:

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/transfer/transfer_medium_trainHalf.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]


In [5]:
history = model.fit(
  train.cache(),
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/50


2022-08-03 15:04:36.086848: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 15:04:36.086930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.1483 - accuracy: 0.5589

2022-08-03 15:04:38.999781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 4s 29ms/step - loss: 1.1483 - accuracy: 0.5589 - val_loss: 0.8005 - val_accuracy: 0.6972
Epoch 2/50
105/105 [==============================] - 3s 29ms/step - loss: 0.6678 - accuracy: 0.7503 - val_loss: 0.5918 - val_accuracy: 0.7764
Epoch 3/50
105/105 [==============================] - 3s 25ms/step - loss: 0.5004 - accuracy: 0.8149 - val_loss: 0.4903 - val_accuracy: 0.8042
Epoch 4/50
105/105 [==============================] - 3s 26ms/step - loss: 0.3951 - accuracy: 0.8577 - val_loss: 0.4262 - val_accuracy: 0.8292
Epoch 5/50
105/105 [==============================] - 3s 26ms/step - loss: 0.3205 - accuracy: 0.8893 - val_loss: 0.3974 - val_accuracy: 0.8458
Epoch 6/50
105/105 [==============================] - 3s 26ms/step - loss: 0.2641 - accuracy: 0.9125 - val_loss: 0.3602 - val_accuracy: 0.8639
Epoch 7/50
105/105 [==============================] - 3s 26ms/step - loss: 0.2136 - accuracy: 0.9307 - val_loss: 0.3325 - val_accuracy: 0.8806
Epoch 8/50

In [6]:
model = keras.models.load_model("../../models/transfer/transfer_medium_trainHalf.h5")
model.evaluate(test)

 7/23 [========>.....................] - ETA: 0s - loss: 0.5684 - accuracy: 0.9107

2022-08-03 15:06:53.910255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 18ms/step - loss: 0.4556 - accuracy: 0.9125


[0.4556175470352173, 0.9125000238418579]

In [ ]:
# Plotting loss and accuracy graphs
## Plotting loss
plot_loss(history)
## Plotting accuracy
plot_acc(history)

# compare with untrained

In [7]:
untrained_model = keras.models.load_model("../../models/zenodo/m3_medium.h5")
untrained_model.evaluate(test)

 7/23 [========>.....................] - ETA: 0s - loss: 0.2139 - accuracy: 0.9420

2022-08-03 15:07:00.243229: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 17ms/step - loss: 0.1695 - accuracy: 0.9542


[0.16947980225086212, 0.9541667103767395]

# All Trainable Layers Transfer Learning - start from nonrandom weights

In [8]:
vtid2_allTrainable = keras.models.load_model('../../models/vtid2/m2_medium.h5')

x = vtid2_allTrainable.layers[-2].output
outputs = keras.layers.Dense(6, activation='softmax', name='classifier')(x)
all_trainable_model = keras.Model(vtid2_allTrainable.inputs, outputs)
all_trainable_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
all_trainable_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                       

In [9]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/transfer/transfer_medium_trainFull.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

In [10]:
history = all_trainable_model.fit(
  train.cache(),
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/50


2022-08-03 15:07:50.790065: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.3917 - accuracy: 0.4435

2022-08-03 15:07:55.784528: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 6s 48ms/step - loss: 1.3917 - accuracy: 0.4435 - val_loss: 0.9300 - val_accuracy: 0.6597
Epoch 2/50
105/105 [==============================] - 5s 45ms/step - loss: 0.8434 - accuracy: 0.6893 - val_loss: 0.6026 - val_accuracy: 0.7792
Epoch 3/50
105/105 [==============================] - 5s 44ms/step - loss: 0.6202 - accuracy: 0.7771 - val_loss: 0.5129 - val_accuracy: 0.8056
Epoch 4/50
105/105 [==============================] - 5s 44ms/step - loss: 0.4663 - accuracy: 0.8327 - val_loss: 0.3790 - val_accuracy: 0.8639
Epoch 5/50
105/105 [==============================] - 5s 46ms/step - loss: 0.3718 - accuracy: 0.8652 - val_loss: 0.3490 - val_accuracy: 0.8792
Epoch 6/50
105/105 [==============================] - 5s 45ms/step - loss: 0.3243 - accuracy: 0.8842 - val_loss: 0.2972 - val_accuracy: 0.8931
Epoch 7/50
105/105 [==============================] - 5s 47ms/step - loss: 0.2436 - accuracy: 0.9095 - val_loss: 0.3804 - val_accuracy: 0.8833
Epoch 8/50

In [11]:
trainable_model = keras.models.load_model("../../models/transfer/transfer_medium_trainFull.h5")
trainable_model.evaluate(test)

 4/23 [====>.........................] - ETA: 0s - loss: 0.4159 - accuracy: 0.9297

2022-08-03 15:10:31.406374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 19ms/step - loss: 0.4067 - accuracy: 0.9208


[0.4067090153694153, 0.9208333492279053]

In [ ]:
# Plotting loss and accuracy graphs
## Plotting loss
plot_loss(history)
## Plotting accuracy
plot_acc(history)